<a href="https://colab.research.google.com/github/Debmalya-prog/ISS_tracking/blob/main/ISS_location.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install skyfield
from skyfield.api import load, wgs84
import pytz
import pylab as pl
import matplotlib.pyplot as plt
ts = load.timescale()

In [ ]:
india = pytz.timezone('Asia/Kolkata')
kolkata = wgs84.latlon(+28.70, +77.10, elevation_m=100)

In [ ]:
stations_url = 'http://celestrak.com/NORAD/elements/stations.txt'
satellites = load.tle_file(stations_url)
print('Loaded', len(satellites), 'satellites')

Loaded 65 satellites


In [ ]:
by_name = {sat.name: sat for sat in satellites}
satellite = by_name['ISS (ZARYA)']
print(satellite)
print(satellite.epoch.utc_jpl())
print(satellite)

ISS (ZARYA) catalog #25544 epoch 2022-03-20 10:01:58 UTC
A.D. 2022-Mar-20 10:01:58.0184 UTC
ISS (ZARYA) catalog #25544 epoch 2022-03-20 10:01:58 UTC


In [ ]:
t = ts.now()
geocentric = satellite.at(t)
subpoint = wgs84.subpoint(geocentric)
print(f'Latitude:  {subpoint.latitude.degrees:.2f}')
print(f'Longitude: {subpoint.longitude.degrees:.2f}')
print(f"Elevation {int(subpoint.elevation.km)} (km) {int(subpoint.elevation.km*0.621371)} (mi)")
print(f"https://earth.google.com/web/search/{subpoint.latitude.degrees:.2f},{subpoint.longitude.degrees:.2f}")

Latitude:  20.23
Longitude: -6.04
Elevation 418 (km) 260 (mi)
https://earth.google.com/web/search/20.23,-6.04


In [ ]:
difference = satellite - kolkata
topocentric = difference.at(t)
alt, az, distance = topocentric.altaz()
print(f"alt {alt}, az {az} distance {int(distance.km)} (km) {int(distance.km*0.621371):,} (mi)")

alt -34deg 56' 26.2", az 285deg 02' 16.6" distance 8000 (km) 4,971 (mi)


In [ ]:
t0 = ts.utc(2022, 3, 19)
t1 = ts.utc(2022, 3, 23)
t, events = satellite.find_events(kolkata, t0, t1, altitude_degrees=10.0)
for ti, event in zip(t, events):
    name = ('rise above 10°', 'culminate', 'set below 10°')[event]
    print(ti.utc_strftime('%Y %b %d %H:%M:%S'), name)

2022 Mar 19 11:51:43 rise above 10°
2022 Mar 19 11:54:03 culminate
2022 Mar 19 11:56:23 set below 10°
2022 Mar 19 13:27:45 rise above 10°
2022 Mar 19 13:30:39 culminate
2022 Mar 19 13:33:34 set below 10°
2022 Mar 19 20:02:24 rise above 10°
2022 Mar 19 20:04:23 culminate
2022 Mar 19 20:06:21 set below 10°
2022 Mar 19 21:38:06 rise above 10°
2022 Mar 19 21:41:16 culminate
2022 Mar 19 21:44:25 set below 10°
2022 Mar 20 12:39:27 rise above 10°
2022 Mar 20 12:42:43 culminate
2022 Mar 20 12:46:01 set below 10°
2022 Mar 20 20:50:05 rise above 10°
2022 Mar 20 20:53:27 culminate
2022 Mar 20 20:56:49 set below 10°
2022 Mar 21 11:51:31 rise above 10°
2022 Mar 21 11:54:51 culminate
2022 Mar 21 11:58:12 set below 10°
2022 Mar 21 20:02:19 rise above 10°
2022 Mar 21 20:05:34 culminate
2022 Mar 21 20:08:48 set below 10°
2022 Mar 21 21:41:30 rise above 10°
2022 Mar 21 21:41:59 culminate
2022 Mar 21 21:42:28 set below 10°
2022 Mar 22 11:03:59 rise above 10°
2022 Mar 22 11:07:02 culminate
2022 Mar 22 11:

In [ ]:
eph = load('de421.bsp')
sun, earth = eph['sun'], eph['earth']
horizon=10
def sun_pos(t, topopos):
  obs = earth + kolkata
  astrometric = obs.at(t).observe(sun)
  apparent = obs.at(t).observe(sun).apparent()
  alt, az, dist = apparent.altaz()
  return alt, az, dist

In [ ]:
for ti, event in zip(t, events):
    name = (f'rise above {horizon}°', 'highest point', f'set below {horizon}°')[event]
    sunalt, sunaz, sundist = sun_pos(ti, kolkata)
    if -20 <= sunalt.degrees <= -6:
      print(ti.utc_jpl(), name)

A.D. 2022-Mar-19 13:27:44.9040 UTC rise above 10°
A.D. 2022-Mar-19 13:30:38.8063 UTC highest point
A.D. 2022-Mar-19 13:33:33.7015 UTC set below 10°
